In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
# Define the directory containing your data
DATA_DIR = "data/Human/Omega-N-methylarginine/" # MODIFY if your path is different

# Define the exact filenames for your simplified CSVs
POS_FILE = os.path.join(DATA_DIR, "human_positive_omega-n-methylarginine_sequences.csv") # Or your positive filename
NEG_FILE = os.path.join(DATA_DIR, "human_negative_omega-n-methylarginine_sequences.csv") # Or your negative filename

try:
    # Load positive data, potentially skipping the unnamed index column if present
    positive_df_raw = pd.read_csv(POS_FILE)
    if positive_df_raw.columns[0].startswith('Unnamed'):
         positive_df_raw = pd.read_csv(POS_FILE, index_col=0)

    # Load negative data
    negative_df_raw = pd.read_csv(NEG_FILE)
    if negative_df_raw.columns[0].startswith('Unnamed'):
         negative_df_raw = pd.read_csv(NEG_FILE, index_col=0)

    print(f"Successfully loaded positive data: {positive_df_raw.shape}")
    print(f"Successfully loaded negative data: {negative_df_raw.shape}")
    print("\nPositive Data Columns:", positive_df_raw.columns.tolist())
    print("Negative Data Columns:", negative_df_raw.columns.tolist())

except FileNotFoundError as e:
    print(f"Error loading file: {e}")
    print(f"Ensure the CSV files exist in the specified path: '{DATA_DIR}'")
    # exit() # Or handle error appropriately in a notebook
except Exception as e:
    print(f"An error occurred during loading: {e}")

Successfully loaded positive data: (987, 5)
Successfully loaded negative data: (56708, 5)

Positive Data Columns: ['Entry', 'Organism', 'Modified residue', 'Modified_position', 'Positive_sequence']
Negative Data Columns: ['Entry', 'Organism', 'Modified residue', 'Negative_R_position', 'Negative_sequence']


In [3]:
print("Preparing and cleaning data...")

pos_seq_col_raw = 'Positive_sequence' # Column containing the positive sequence
if pos_seq_col_raw not in positive_df_raw.columns:
    raise ValueError(f"Column '{pos_seq_col_raw}' not found in positive data.")

positive_data = positive_df_raw[[pos_seq_col_raw]].copy() # Keep only the sequence column for now
positive_data.rename(columns={pos_seq_col_raw: 'Sequence'}, inplace=True)
print(f"Positive data prepared. Shape: {positive_data.shape}")

Preparing and cleaning data...
Positive data prepared. Shape: (987, 1)


In [4]:
neg_seq_col_raw = 'Negative_sequence' # Column containing the negative sequence
if neg_seq_col_raw not in negative_df_raw.columns:
     raise ValueError(f"Column '{neg_seq_col_raw}' not found in negative data.")

negative_data = negative_df_raw[[neg_seq_col_raw]].copy() # Keep only the sequence column for now
negative_data.rename(columns={neg_seq_col_raw: 'Sequence'}, inplace=True)
print(f"Negative data prepared. Shape: {negative_data.shape}")


Negative data prepared. Shape: (56708, 1)


In [5]:
# Add length column to both dataframes for filtering
positive_data['Length'] = positive_data['Sequence'].astype(str).str.len()
negative_data['Length'] = negative_data['Sequence'].astype(str).str.len()

print("\nSample positive data after prep:")
print(positive_data.head())
print("\nSample negative data after prep:")
print(negative_data.head())


Sample positive data after prep:
      Sequence  Length
0  YQKAARGGGAA      11
1  RAPGPRGSYLG      11
2  GYSAGRGIYSR      11
3  STKRGRGGGEA      11
4  CSLEIRKPGPF      11

Sample negative data after prep:
      Sequence  Length
0  LALMERTGYSM      11
1  QENGQRKYGGP      11
2  GPHPQRGCEVF      11
3  VGKIPRDVYED      11
4  FEAVGRIYELR      11


In [6]:
positive_data.head(3)

,Sequence,Length
0,YQKAARGGGAA,11
1,RAPGPRGSYLG,11
2,GYSAGRGIYSR,11


In [7]:
negative_data.head(3)

,Sequence,Length
0,LALMERTGYSM,11
1,QENGQRKYGGP,11
2,GPHPQRGCEVF,11


In [8]:
# Standard amino acids (canonical 20) in fixed order.
amino_acids = list("ACDEFGHIKLMNPQRSTVWY")  # This order is fixed for all sequences.
aa_to_index = {aa: i for i, aa in enumerate(amino_acids)}

def one_hot_encode(sequence, aa_to_index):

    # Initialize a zero matrix for one-hot encoding.
    encoding = np.zeros((len(sequence), len(aa_to_index)), dtype=int)
    for i, aa in enumerate(sequence):
        encoding[i, aa_to_index[aa]] = 1
    return encoding

# Apply one-hot encoding to the positive sequences.
positive_data['one_hot'] = positive_data['Sequence'].apply(lambda seq: one_hot_encode(seq, aa_to_index))
# Print the shape of one hot encoding for the first positive sequence.
print("Positive one hot shape:", positive_data['one_hot'].iloc[0].shape)

# Apply one-hot encoding to the negative sequences.
negative_data['one_hot'] = negative_data['Sequence'].apply(lambda seq: one_hot_encode(seq, aa_to_index))
# Print the shape of one hot encoding for the first negative sequence.
print("Negative one hot shape:", negative_data['one_hot'].iloc[0].shape)

Positive one hot shape: (11, 20)
Negative one hot shape: (11, 20)


In [9]:
from math import log2

# Define a function to calculate Shannon entropy for a sequence.
def shannon_entropy(sequence):
    """
    Calculate the Shannon entropy of a given amino acid sequence.
    """
    length = len(sequence)
    # Count frequency of each amino acid in the sequence.
    counts = {}
    for aa in sequence:
        counts[aa] = counts.get(aa, 0) + 1
    
    # Compute entropy.
    entropy = 0.0
    for count in counts.values():
        p = count / length
        entropy -= p * log2(p)
    return entropy

# Apply the function to create a new feature column for both datasets.
positive_data['shannon_entropy'] = positive_data['Sequence'].apply(shannon_entropy)
negative_data['shannon_entropy'] = negative_data['Sequence'].apply(shannon_entropy)

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

all_sequences = pd.concat([positive_data['Sequence'], negative_data['Sequence']])

# 2. Define the TF-IDF vectorizer for character-level features.
#    Using analyzer='char' and ngram_range=(1,1) ensures each amino acid is treated as a separate feature.
vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(1, 1))
vectorizer.fit(all_sequences)

# 3. Transform the sequences in each dataframe.
positive_tfidf = vectorizer.transform(positive_data['Sequence'])
negative_tfidf = vectorizer.transform(negative_data['Sequence'])

# 4. Convert the TF-IDF sparse matrices to dense arrays and add as a new column in the dataframes.
#    Each element in the new column 'tfidf' is a vector standardized based on the common vocabulary.
positive_data['tfidf'] = list(positive_tfidf.toarray())
negative_data['tfidf'] = list(negative_tfidf.toarray())

# Print the shape of the first TF-IDF feature in each dataframe.
print("Positive TF-IDF feature shape:", positive_data['tfidf'].iloc[0].shape)
print("Negative TF-IDF feature shape:", negative_data['tfidf'].iloc[0].shape)


Positive TF-IDF feature shape: (20,)
Negative TF-IDF feature shape: (20,)


In [11]:
amino_acids = list("ACDEFGHIKLMNPQRSTVWY")
aa_to_index = {aa: i for i, aa in enumerate(amino_acids)}
num_dipeptides = len(amino_acids) * len(amino_acids)  # 400

def dipeptide_composition(sequence, aa_to_index):
    """
    Calculate the dipeptide composition for a given sequence.
    Returns a 400-dimensional vector where each element is the normalized count
    of the corresponding dipeptide.
    """
    comp = np.zeros(num_dipeptides, dtype=float)
    n = len(sequence)
    if n < 2:
        return comp  # Not enough characters for a dipeptide.
    
    total_dipeptides = n - 1
    for i in range(total_dipeptides):
        aa1 = sequence[i]
        aa2 = sequence[i+1]
        if aa1 in aa_to_index and aa2 in aa_to_index:
            idx = aa_to_index[aa1] * len(aa_to_index) + aa_to_index[aa2]
            comp[idx] += 1
        else:
            # Handle unknown characters if necessary.
            continue
    # Normalize the vector so that the sum equals 1 (frequency distribution).
    comp = comp / total_dipeptides
    return comp

# Apply the function to the positive sequences.
positive_data['dpc'] = positive_data['Sequence'].apply(lambda seq: dipeptide_composition(seq, aa_to_index))
# Print the shape of the DPC vector for the first positive sequence.
print("Positive DPC feature shape:", positive_data['dpc'].iloc[0].shape)

# Apply the function to the negative sequences.
negative_data['dpc'] = negative_data['Sequence'].apply(lambda seq: dipeptide_composition(seq, aa_to_index))
# Print the shape of the DPC vector for the first negative sequence.
print("Negative DPC feature shape:", negative_data['dpc'].iloc[0].shape)

Positive DPC feature shape: (400,)
Negative DPC feature shape: (400,)


In [12]:
def compute_pseaac(sequence, lambda_value=5, w=0.05):
    
    # Define the canonical amino acids and filter the sequence.
    aa_list = "ACDEFGHIKLMNPQRSTVWY"
    valid_set = set(aa_list)
    seq = ''.join([aa for aa in sequence.upper() if aa in valid_set])
    L = len(seq)
    if L == 0:
        return np.array([np.nan]*(20+lambda_value))
    
    # Conventional Amino Acid Composition (AAC): 20 features.
    AAC = np.zeros(20)
    for aa in seq:
        idx = aa_list.index(aa)
        AAC[idx] += 1
    AAC = AAC / L  # normalize by sequence length

    # Define three physicochemical properties.
    # 1. Hydrophobicity (using a common scale)
    hydrophobicity = {
        'A': 0.62, 'R': -2.53, 'N': -0.78, 'D': -0.90, 'C': 0.29,
        'E': -0.74, 'Q': -0.85, 'G': 0.48, 'H': -0.40, 'I': 1.38,
        'L': 1.06, 'K': -1.50, 'M': 0.64, 'F': 1.19, 'P': 0.12,
        'S': -0.18, 'T': -0.05, 'W': 0.81, 'Y': 0.26, 'V': 1.08
    }
    # 2. Hydrophilicity (using the Hopp-Woods scale)
    hydrophilicity = {
        'A': -0.5, 'R': 3.0, 'N': 0.2, 'D': 3.0, 'C': -1.0,
        'E': 3.0, 'Q': 0.2, 'G': 0.0, 'H': -0.5, 'I': -1.8,
        'L': -1.8, 'K': 3.0, 'M': -1.3, 'F': -2.5, 'P': 0.0,
        'S': 0.3, 'T': -0.4, 'W': -3.4, 'Y': -2.3, 'V': -1.5
    }
    # 3. Side-chain mass (approximate molecular weights)
    mass = {
        'A': 89.09, 'R': 174.20, 'N': 132.12, 'D': 133.10, 'C': 121.15,
        'E': 147.13, 'Q': 146.15, 'G': 75.07, 'H': 155.16, 'I': 131.17,
        'L': 131.17, 'K': 146.19, 'M': 149.21, 'F': 165.19, 'P': 115.13,
        'S': 105.09, 'T': 119.12, 'W': 204.23, 'Y': 181.19, 'V': 117.15
    }
    
    # Helper function: Normalize a property dictionary.
    def normalize_property(prop_dict):
        values = np.array([prop_dict[aa] for aa in aa_list])
        mean = np.mean(values)
        std = np.std(values)
        return {aa: (prop_dict[aa] - mean) / std for aa in aa_list}

    norm_hydro = normalize_property(hydrophobicity)
    norm_hphil = normalize_property(hydrophilicity)
    norm_mass  = normalize_property(mass)
    
    # Compute sequence-order correlation factors theta for lags 1 to lambda_value.
    theta = np.zeros(lambda_value)
    if L <= 1:
        theta = np.zeros(lambda_value)
    else:
        for l in range(1, lambda_value+1):
            if L - l <= 0:
                theta[l-1] = 0
            else:
                sum_corr = 0.0
                for i in range(L - l):
                    diff1 = norm_hydro[seq[i]] - norm_hydro[seq[i+l]]
                    diff2 = norm_hphil[seq[i]] - norm_hphil[seq[i+l]]
                    diff3 = norm_mass[seq[i]] - norm_mass[seq[i+l]]
                    sum_corr += (diff1**2 + diff2**2 + diff3**2) / 3.0
                theta[l-1] = sum_corr / (L - l)
    
    # Calculate the normalization factor.
    norm_factor = 1 + w * np.sum(theta)
    
    # Construct the PseAAC vector: first 20 are AAC, next lambda_value are the theta terms.
    pseaac_vector = np.concatenate((AAC / norm_factor, (w * theta / norm_factor)))
    return pseaac_vector


positive_data['pseaac'] = positive_data['Sequence'].apply(lambda seq: compute_pseaac(seq, lambda_value=5, w=0.05))
negative_data['pseaac'] = negative_data['Sequence'].apply(lambda seq: compute_pseaac(seq, lambda_value=5, w=0.05))

# Print the shape of the first PseAAC feature in each dataframe.
print("Positive PseAAC feature shape:", positive_data['pseaac'].iloc[0].shape)
print("Negative PseAAC feature shape:", negative_data['pseaac'].iloc[0].shape)


Positive PseAAC feature shape: (25,)
Negative PseAAC feature shape: (25,)


In [13]:
def create_feature_vector(seq_features):
    
    # Extract and flatten each feature:
    # 1. One-hot: a 2D array; flatten it.
    one_hot_vec = np.array(seq_features['one_hot']).flatten()
    
    # 2. Shannon entropy: scalar -> convert to 1D array.
    shannon_vec = np.array([seq_features['shannon_entropy']])
    
    # 3. TF-IDF: assume already a vector.
    tfidf_vec = np.array(seq_features['tfidf'])
    
    # 4. Dipeptide Composition (DPC): vector.
    dpc_vec = np.array(seq_features['dpc'])
    
    # 5. PseAAC: vector.
    pseaac_vec = np.array(seq_features['pseaac'])     
    
    combined_vector = np.concatenate([one_hot_vec, shannon_vec, tfidf_vec, dpc_vec, pseaac_vec])


    total_length = combined_vector.shape[0]
    vector_height = int(np.ceil(total_length / 64))

    padded_length = vector_height * 64

    padded_vector = np.pad(combined_vector, (0, padded_length - total_length), mode='constant', constant_values=0)

    vector = padded_vector.reshape((vector_height, 64))

    pad_height = 64 - vector.shape[0]
    pad_width = 64 - vector.shape[1]

    pad_top = pad_height // 2
    pad_bottom = pad_height - pad_top
    pad_left = pad_width // 2
    pad_right = pad_width - pad_left

    padded_vector = np.pad(vector, ((pad_top, pad_bottom), (pad_left, pad_right)), mode='constant', constant_values=0)
    
    return padded_vector




In [14]:
# Lists to store the feature images for positive and negative sequences.
positive_vector = []
negative_vector = []

# Loop over each row in the positive_data dataframe.
for index, row in positive_data.iterrows():
    seq_features = {
        'one_hot': row['one_hot'],
        'shannon_entropy': row['shannon_entropy'],
        'tfidf': row['tfidf'],
        'dpc': row['dpc'],
        'pseaac': row['pseaac']
    }
    img = create_feature_vector(seq_features)  # You can adjust image_width as needed.
    positive_vector.append(img)


# Loop over each row in the negative_data dataframe.
for index, row in negative_data.iterrows():
    seq_features = {
        'one_hot': row['one_hot'],
        'shannon_entropy': row['shannon_entropy'],
        'tfidf': row['tfidf'],
        'dpc': row['dpc'],
        'pseaac': row['pseaac']
    }
    img = create_feature_vector(seq_features)
    negative_vector.append(img)

# Print the shape of one sample image from each list for verification.
print("Positive vector shape:", positive_vector[0].shape)
print("Negative vector shape:", negative_vector[0].shape)


Positive vector shape: (64, 64)
Negative vector shape: (64, 64)


In [15]:
len(positive_vector), len(negative_vector)

(987, 56708)

In [16]:
pos_labels = np.ones(len(positive_vector))
neg_labels = np.zeros(len(negative_vector))


X = np.vstack([positive_vector, negative_vector])
y = np.concatenate([pos_labels, neg_labels])



In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.exceptions import ConvergenceWarning

import os
import warnings

# Option 1: Set the maximum CPU count (adjust "8" to your desired core count)
os.environ['LOKY_MAX_CPU_COUNT'] = "8"

warnings.filterwarnings("ignore", category=ConvergenceWarning)


# --- Prepare indices for positive and negative samples ---
# Assuming X and y are computed from the previous blocks
# (X : combined feature matrix, y : labels with 1 for positive and 0 for negative)
npos = np.sum(y == 1)
nneg = np.sum(y == 0)

print(f"Total positive samples: {npos}")
print(f"Total negative samples: {nneg}")

# The positive data are always included. For negatives, we split indices into k folds.
# Shuffle all negative indices
rng = np.random.default_rng(seed=42)
neg_indices = np.arange(npos, npos + nneg)  # note: positive are from 0 to npos-1, negatives follow
rng.shuffle(neg_indices)

# Partition the negative indices into folds of size equal to npos
folds = []
num_folds = int(np.ceil(nneg / npos))
for fold in range(num_folds):
    start = fold * npos
    end = start + npos
    fold_neg_idx = neg_indices[start:end]
    folds.append(fold_neg_idx)
    
print(f"Created {len(folds)} folds for negatives (each of size ~{npos}).")

# --- Define classifiers ---
classifiers = {
    "SVM": SVC(probability=False, random_state=42),
    "RandomForest": RandomForestClassifier(random_state=42),
    "KNN": KNeighborsClassifier(),
    "MLP": MLPClassifier(random_state=42, max_iter=300)
}

# --- Result collection ---
results = []

# Iterate over each negative fold. For each fold, the positive samples remain the same.
for i, neg_fold_idx in enumerate(folds, start=1):
    print(f"\nProcessing fold {i}/{len(folds)} with {len(neg_fold_idx)} negative samples...")
    
    # Create subset for the current fold:
    # Positive indices: from 0 to npos-1 (they remain constant)
    pos_idx = np.arange(npos)
    # Combine indices: first positives then negatives from current fold
    current_indices = np.concatenate((pos_idx, neg_fold_idx))
    
    X_fold = X[current_indices]
    y_fold = y[current_indices]
    
    # Optionally shuffle the combined data set (which is balanced or nearly balanced)
    X_train, X_test, y_train, y_test = train_test_split(
        X_fold, y_fold, test_size=0.2, stratify=y_fold, random_state=42
    )
    
    n_train = X_train.shape[0]
    n_test  = X_test.shape[0]

    X_train_flat = X_train.reshape(n_train, -1)
    X_test_flat  = X_test.reshape(n_test,  -1)

    for model_name, clf in classifiers.items():
        # Train the classifier
        clf.fit(X_train_flat, y_train)
        # Predict on the test set
        y_pred = clf.predict(X_test_flat)
        
        # Calculate metrics
        acc  = accuracy_score(y_test, y_pred)
        pre  = precision_score(y_test, y_pred, zero_division=0)
        rec  = recall_score(y_test, y_pred, zero_division=0)
        f1   = f1_score(y_test, y_pred, zero_division=0)
        mcc  = matthews_corrcoef(y_test, y_pred)
        
        # Append results for this fold and classifier
        results.append({
            "Fold": i,
            "Classifier": model_name,
            "Accuracy": acc,
            "Precision": pre,
            "Recall": rec,
            "F1": f1,
            "MCC": mcc
        })
        print(f"Fold {i} - {model_name}: Accuracy: {acc:.3f}, Precision: {pre:.3f}, Recall: {rec:.3f}, F1: {f1:.3f}, MCC: {mcc:.3f}")

# Convert results to a DataFrame and display
results_df = pd.DataFrame(results)
print("\nOverall Results:")
print(results_df)


Total positive samples: 987
Total negative samples: 56708
Created 58 folds for negatives (each of size ~987).

Processing fold 1/58 with 987 negative samples...
Fold 1 - SVM: Accuracy: 0.790, Precision: 0.831, Recall: 0.726, F1: 0.775, MCC: 0.584
Fold 1 - RandomForest: Accuracy: 0.767, Precision: 0.790, Recall: 0.726, F1: 0.757, MCC: 0.536
Fold 1 - KNN: Accuracy: 0.706, Precision: 0.687, Recall: 0.756, F1: 0.720, MCC: 0.415
Fold 1 - MLP: Accuracy: 0.772, Precision: 0.802, Recall: 0.721, F1: 0.759, MCC: 0.547

Processing fold 2/58 with 987 negative samples...
Fold 2 - SVM: Accuracy: 0.737, Precision: 0.736, Recall: 0.736, F1: 0.736, MCC: 0.473
Fold 2 - RandomForest: Accuracy: 0.722, Precision: 0.723, Recall: 0.716, F1: 0.719, MCC: 0.443
Fold 2 - KNN: Accuracy: 0.706, Precision: 0.667, Recall: 0.822, F1: 0.736, MCC: 0.425
Fold 2 - MLP: Accuracy: 0.714, Precision: 0.698, Recall: 0.751, F1: 0.724, MCC: 0.429

Processing fold 3/58 with 987 negative samples...
Fold 3 - SVM: Accuracy: 0.752, 

C:\Users\shubh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\neural_network\_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


Fold 19 - MLP: Accuracy: 0.742, Precision: 0.768, Recall: 0.690, F1: 0.727, MCC: 0.486

Processing fold 20/58 with 987 negative samples...
Fold 20 - SVM: Accuracy: 0.765, Precision: 0.795, Recall: 0.711, F1: 0.751, MCC: 0.532
Fold 20 - RandomForest: Accuracy: 0.757, Precision: 0.773, Recall: 0.726, F1: 0.749, MCC: 0.515
Fold 20 - KNN: Accuracy: 0.752, Precision: 0.726, Recall: 0.807, F1: 0.764, MCC: 0.507


In [ ]:
# get average of the results. ignore the fold column
results_df_avg = results_df.groupby('Classifier').mean().reset_index()
results_df_avg = results_df_avg.drop(columns=['Fold'])

results_df_avg = results_df_avg.round(3)
print("\nAverage Results:")
print(results_df_avg)


Average Results:
     Classifier  Accuracy  Precision  Recall     F1    MCC
0           KNN     0.861      0.967   0.749  0.844  0.741
1           MLP     0.925      0.928   0.922  0.925  0.849
2  RandomForest     0.885      0.874   0.903  0.888  0.770
3           SVM     0.925      0.899   0.958  0.927  0.851
